In [49]:
import os
import shutil
import re

In [17]:
VERBOSE = True

In [245]:
def modernize(script, output='ps'):
    modern = []
    ps_name = None
    
    # Copy the header comments and add a command to set the mode 
    # to modern after the header
    for l, line in enumerate(script):
        line = line.strip()
        if not line or line[0] != '#':
            break
        modern.append(line)
    modern.append('gmt set GMT_RUNMODE modern')
    
    # Parse the rest of the script
    for line in script[l:]:
        line = line.strip()
        
        # Check if this line defines a .ps file name variable
        ps_var_def = re.search(r'^ps=.+\.ps$', line)
        if ps_var_def is not None:
            ps_definition = ps_var_def.group()
            ps_file_name = ps_definition.split('=')[1]
            ps_name = os.path.splitext(ps_file_name)[0]
            line = os.path.splitext(ps_definition)[0]
            
        # Check if redirecting to the $ps variable and strip it out
        redirect_to_ps = re.search(r'.+>+ *\$ps$', line)
        if redirect_to_ps is not None:
            redirection = re.search(r'>+ *\$ps$', line)
            line = line.replace(redirection.group(), '').strip()
            
        # Remove -O -K -R -J
        okrj = re.findall(r'(?<= )-[KORJ](?: +|$)', line)
        if okrj:
            for match in okrj:
                line = line.replace(match, '')
            line = line.strip()
        
        modern.append(line)
        
    # Only add a psconvert call if this script defined a ps variable
    if ps_name is not None:
        if output == 'ps':
            args = '-Tp'
        elif output == 'pdf':
            args = '-Tf -P -A'
        else:
            assert False
        modern.append('') 
        modern.append('gmt psconvert {} -F$ps'.format(args))
        modern.append('')         
        
    return modern

In [246]:
input_dir = 'test'
output_dir = 'test_modern'

In [247]:
if os.path.exists(output_dir):
    print('Aborting: Output directory already exists.')

Aborting: Output directory already exists.


In [248]:
def info(*args, **kwargs):
    """
    Print message to the screen according to set verbosity.
    """
    if VERBOSE:
        print(*args, **kwargs)

In [250]:
!rm -rf $output_dir

for base, dirs, files in os.walk(input_dir):
    info('Base dir:', base)
    base_output = base.replace(input_dir, output_dir)
    os.mkdir(base_output)
    scripts = set(f for f in files if os.path.splitext(f)[-1] == '.sh')
    not_scripts = set(files).difference(scripts)
    for file in not_scripts:
        shutil.copy(os.path.join(base, file), base_output)
    for script in scripts:
        modern_script = os.path.join(base_output, script)
        old_script = os.path.join(base, script)
        info('  Converting:', old_script, ' --> ', modern_script)
        with open(old_script) as f:
            old_content = f.readlines()
        modern_content = modernize(old_content, output='ps')
        with open(modern_script, 'w') as f:
            f.write('\n'.join(modern_content))

Base dir: test
Base dir: test/grdcontour
  Converting: test/grdcontour/polcont.sh  -->  test_modern/grdcontour/polcont.sh
  Converting: test/grdcontour/polcontr.sh  -->  test_modern/grdcontour/polcontr.sh
  Converting: test/grdcontour/grdcontour.sh  -->  test_modern/grdcontour/grdcontour.sh
Base dir: test/psxy
  Converting: test/psxy/wedges.sh  -->  test_modern/psxy/wedges.sh
  Converting: test/psxy/plot_symbols.sh  -->  test_modern/psxy/plot_symbols.sh
  Converting: test/psxy/vector.sh  -->  test_modern/psxy/vector.sh
Base dir: test/deeper
Base dir: test/deeper/pscoast
  Converting: test/deeper/pscoast/pscoast_JE3.sh  -->  test_modern/deeper/pscoast/pscoast_JE3.sh
  Converting: test/deeper/pscoast/pscoast_JM.sh  -->  test_modern/deeper/pscoast/pscoast_JM.sh


In [281]:
re.findall(r'(?:(?<= )|^)-[KORJ](?: +|$)', '-O -T+d0.1i/0.02i e-O -K -R -Rbla -J')

['-O ', '-K ', '-R ', '-J']